Le code source de ce cours est disponible sur https://github.com/asardell/statistique-python

<font size="6">  Test d'indépendance 🎲 avec Python </font>

Les tests d'indépendances permettent de définir s'il existe un lien entre deux variables. Il existe différent test d'indépence, en voici quelques exemples :

* Test indépendance entre deux variables quantitatives / Test de corrélation Pearson
* Test d'indépendance entre deux variables qualitatives / Test du Chi²
* Test d'indépendance entre une variable qualitative et une quantitative / Test de Fisher avec l'analyse de la variance (ANOVA)

<br>

Dans une démarche d'un projet de machine learning, les test d'indépendance permettent d'exclure des variables explicatives potentiellement non porteuses d'information.

Librairies utilisées

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd

from scipy.stats import pearsonr
from scipy.stats import bartlett
from scipy.stats import shapiro
from scipy.stats import chi2_contingency
from scipy.stats import kendalltau, spearmanr

from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_regression, make_circles

import statsmodels.stats.multicomp as multi 
import statsmodels.api as sm
from statsmodels.formula.api import ols


# Test de corrélation Pearson

L’intérêt des tests de corrélation est d’apporter plus de pertinence et fiabilité aux coefficients de corrélation. Il existe différents test de corrélation, nous utilisons celui de Pearson.

On travaille avec le jeu de données fromage 🧀 disponible en [cliquant ici](https://github.com/asardell/statistique-python/tree/master/Dataset)

In [ ]:
df = pd.read_table("../Dataset/fromage.txt", index_col=0)
df.head()

Avant de réaliser des tests d'indépendance, on projette graphiquement les données 2 à 2

In [ ]:
sns.pairplot(df.iloc[:,0:9])

## Matrice des corrélations

Voici la matrice des corrélations des variables du fichier fromage.

In [ ]:
sns.set(rc={'figure.figsize':(10,4)})

df_corr = df.corr()

ax = sns.heatmap(df_corr, xticklabels = df_corr.columns , 
                 yticklabels = df_corr.columns, cmap = 'coolwarm')

L'intérêt des tests de corrélation est d'apporter plus de pertinence et fiabilité aux coefficients de corrélation. Il existe différents test de corrélation, nous utilisons celui de Pearson.

In [ ]:
from scipy.stats import pearsonr

On pose les hypothèses de départ :

* H0 : Variables indépendantes si p-value > 5%
* H1 : Variables non indépendantes si p-value < 5%

### Lipides vs Magnesium

La première sortie correspond au coefficient de corrélation, la seconde à la p-value (ou probabilité critique)

In [ ]:
pearsonr(df.lipides, df.magnesium)

~H0 : Variables indépendantes si p-value > 5%~
<br> H1 : Variables non indépendantes si p-value < 5%

### Sodium vs Retinol

In [ ]:
pearsonr(df.sodium, df.retinol)

H0 : Variables indépendantes si p-value > 5%
<br> ~H1 : Variables non indépendantes si p-value < 5%~ <br>
Si on veut rejeter H0 et prendre H1, j'ai 45,5% de chance de me tromper

📢 Les tests statistiques sont trés sensibles à la taille de l'échantillon. 
Un coefficient de corrélation de 0.14 n'aura pas la même significativité sur un échantillon de 29 fromages qu'un échantillon de 319 fromages avec le même coefficient de corrélation.

On construit un daatframe en duppliquant le nombre de lignes

In [ ]:
df_append = df.copy()
df_append.reset_index(inplace=True)
df_append = df_append.append([df_append]*10,ignore_index=True)
df_append.shape

Chaque fromage apparaît plusieurs fois, on a augmenté la taille de l'échantillon

In [ ]:
df_append.Fromages.value_counts().head()

On effectue un autre test de corrélation avec les mêmes variables sur l'échantillon plus grand.

In [ ]:
pearsonr(df_append.sodium, df_append.retinol)

~H0 : Variables indépendantes si p-value > 5%~
<br> H1 : Variables non indépendantes si p-value < 5% <br>

On obtient logiquement le même coefficient de corrélation, mais en revanche, cette fois si la p-value est proche de 0.

###  Matrice des p-values

On effectue un test de corrélation sur chaque variable 2 à 2 en isolant uniquement la p-value

In [ ]:
a = np.empty((len(df.columns),len(df.columns),))
a[:] = np.nan
for i in range(0,len(df.columns)):
    for j in range(0,len(df.columns)):
        a[i,j] = pearsonr(df.iloc[:,i], df.iloc[:,j])[1]

df_pvalue = round(pd.DataFrame(a, columns=df.columns, index = df.columns),5)

On affiche la matrice des corrélations avec un gradiant de couleur

In [ ]:
cm = sns.light_palette("green", as_cmap=True) 

df_pvalue.\
style.background_gradient(cmap=cm).set_precision(2)

💡 Obtient-on les mêmes p-value si on centre et on réduit ?

On centre et on réduit

In [ ]:
from sklearn.preprocessing import StandardScaler
df_CR = StandardScaler().fit_transform(df)
df_CR = pd.DataFrame(df_CR, columns=df.columns)
df_CR.head(3)

On calcule les test de corrélation

In [ ]:
for i in range(0,len(df.columns)):
    for j in range(0,len(df.columns)):
        a[i,j] = pearsonr(df_CR.iloc[:,i], df_CR.iloc[:,j])[1]

df_CR_pvalue = round(pd.DataFrame(a, columns=df.columns, index = df.columns),5)

On affiche la matrice des p-value

In [ ]:
cm = sns.light_palette("green", as_cmap=True) 

df_CR_pvalue.\
style.background_gradient(cmap=cm).set_precision(2)

💡 On obtient bien les mêmes p-value si on centre et on réduit

## ⛔ Cas de relation non linéaire

Les différents coefficients de corrélation sont beaucoup plus adaptés aux relation linéaire. C’est pourquoi il est important de toujours visualiser les distributions.

Plus d'infos [ici](http://grasland.script.univ-paris-diderot.fr/STAT98/stat98_6/stat98_6.htm)

In [ ]:
from sklearn.datasets import make_regression, make_circles
from scipy.stats import kendalltau, spearmanr

### Cas d'une relation linéaire et monotone

In [ ]:
X, y = make_regression(n_samples=1000, n_features=1,
                                      n_informative=1, noise=50, random_state=0)

plt.scatter(X, y, edgecolor='k', marker='.')
x = pd.DataFrame(X, columns = ['x'])
y = pd.DataFrame(y, columns = ['y'])

In [ ]:
print("Pearson " + str(pearsonr(x.x, y.y)))
print(kendalltau(x.x, y.y))
print(spearmanr(x.x, y.y))

### Cas d'une relation non-linéaire et non-monotone

La parabole 

In [ ]:
X_hyperbole = X[X.y < 0]
fig = plt.figure(figsize=(9, 8))
ax = plt.subplot(221)
ax.scatter(X_hyperbole.x, X_hyperbole.y, s=50, edgecolor='k')
plt.show()

print("Pearson " + str(pearsonr(X_hyperbole.x, X_hyperbole.y)))
print(kendalltau(X_hyperbole.x, X_hyperbole.y))
print(spearmanr(X_hyperbole.x, X_hyperbole.y))

Le cercle

In [ ]:
X = make_circles(n_samples=100,factor=0.99, random_state=0, noise=0.05)[0]
X = pd.DataFrame(X, columns=['x','y'])
fig = plt.figure(figsize=(9, 8))
ax = plt.subplot(221)
ax.scatter(X.x, X.y, s=50, edgecolor='k')
plt.show()

print("Pearson " + str(pearsonr(X.x, X.y)))
print(kendalltau(X.x, X.y))
print(spearmanr(X.x, X.y))

# Test du Khi²

L'intérêt du test du Khi² est de mesurer l'indépendance entre deux variables qualitatives à partir du tableau de contigence.

On travaille sur le jeu de données Titanic 🧊⛴ disponible en [cliquant ici](https://github.com/asardell/statistique-python/tree/master/Dataset)

In [ ]:
df = pd.read_csv("../Dataset/Titanic.csv", index_col=0)
df.head()

In [ ]:
df_count = pd.crosstab(df.Survived, df.PClass)
df_count

On pose les hypothèses de départ :

* H0 : Variables indépendantes si p-value > 5%
* H1 : Variables non indépendantes si p-value < 5%

### Survived vs PClass

In [ ]:
from scipy.stats import chi2_contingency
Khi2_obs, p_value, ddl, effectif_theorique = chi2_contingency(df_count)

In [ ]:
p_value

H0 : Variables indépendantes si p-value > 5%
<br> ~H1 : Variables non indépendantes si p-value < 5%~


### Exemple sur données fictives

In [ ]:
obs = np.array([[693,886,534,153], [597,696,448,95]])
chi2_contingency(obs)

H0 : Variables indépendantes si p-value > 5%
<br> ~H1 : Variables non indépendantes si p-value < 5%~ <br>
Si on veut rejeter H0 et prendre H1, j'ai 10,9% de chance de me tromper

Le 10,9% correspond à la probabilité de rejeter à tord H0. Comment la calculer ?

 Lecture dans la table du Chi2

In [ ]:
from scipy.stats import chi2
J = df = np.arange(1,5,1)
I = np.arange(0.05,0.15,0.005)

a = np.empty((len(J),len(I)))
a[:] = np.nan

for i in range(0,len(I)):
    for j in range(0,len(J)):
        a[j,i] = chi2.isf(I[i], J[j])
        
df_chi2 = round(pd.DataFrame(a, columns=I, index = J),5)
df_chi2

On cherche quelle est la probabilité critique pour laquelle **Khi2_obs** < Khi2_max de la table sur la ligne correspond à notre nombre de degré de liberté **ddl**

### 📢 Taille de l'échantillon

Les tests d'indépendance sont trés sensibles à la taille des échantillons. Ici on divise par 100 pour avoir des effectifs faibles mais en conservant les répartitions.

In [ ]:
chi2_contingency(obs/100)

H0 : Variables indépendantes si p-value > 5%
<br> ~H1 : Variables non indépendantes si p-value < 5%~ <br>

Ici on multiplie par 100 pour avoir des effectifs grands mais en conservant les répartitions.

In [ ]:
chi2_contingency(obs*100)

~H0 : Variables indépendantes si p-value > 5%~
<br> H1 : Variables non indépendantes si p-value < 5%

# ANOVA

## Anova à 1 facteur

On effectue une analyse de variance pour mesurer l’indépendance entre une variable qualitative et une quantitative.

Exemple sur le dataset Hotdogs 🌭 disponible en [cliquant ici](https://github.com/asardell/statistique-python/tree/master/Dataset).

In [ ]:
df = pd.read_csv("../Dataset/Hotdogs.csv", sep = ";")
df.head()

In [ ]:
df.Type.unique()

On va tester l’indépendance entre la variable qualitative Type et la variable quantitatives Calories.

In [ ]:
plt.subplots(figsize=(20,4))
ax = sns.boxplot(x="Calories", y="Type", data=df)

Dans une ANOVA, on cherche à déterminer si les moyennes des groupes sont significativement différentes. On pose donc :

* H0 : Les moyennes de chaque groupe sont égales si p-value > 5%
* H1 : Les moyennes de chaque groupe ne sont pas toutes égales si p-value < 5%

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
model = ols('Calories ~ Type', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

~H0 : Les moyennes de chaque groupe sont égales si p-value > 5%~
<br> H1 : Les moyennes de chaque groupe ne sont pas toutes égales < 5%

Quand on dispose d’un petit échantillon, la pertinence de ce test repose sur la validation de plusieurs hypothèses :

* l’indépendance entre les échantillons de chaque groupe
* l’égalité des variances que l’on peut verifier avec un test de Bartlett.
* la normalité des résidus avec un test de Shapiro.

### L'indépendance

L’indépendance est une des 3 conditions de validité d’une ANOVA. Seul le contexte de l’étude permet de s’assurer de l’indépendance entre les échantillons de chaque groupe (ici beef, poultry, chicken.)

### L’égalité des variances

On parle aussi d’homoscédasticité. C’est une des 3 conditions de validité d’une ANOVA. On cherche à démontrer que les variances de chaque groupe sont égales. Dans un boxplot, l’amplitude des boîtes traduit graphiquement l’égalité des variances.

In [ ]:
plt.subplots(figsize=(20,4))
ax = sns.boxplot(x="Calories", y="Type", data=df)

In [ ]:
df.groupby("Type")['Calories'].agg('var')

Mais c’est le test de bartlett qui permet de tester si les variances sont significativement différentes ou non avec :

* H0 : Les variances de chaque groupe sont égales si p-value > 5%
* H1 : Les variances de chaque groupe ne sont pas toutes égales < 5%

In [ ]:
bartlett(df.Calories[df.Type == 'Beef'],
        df.Calories[df.Type == 'Meat'],
        df.Calories[df.Type == 'Poultry'])

* H0 : Les variances de chaque groupe sont égales si p-value > 5%
* ~H1 : Les variances de chaque groupe ne sont pas toutes égales < 5%~

Les variances de chaque groupe sont égales. La deuxième condition pour effectuer une anova est validée.

### Normalité des résidus

C’est une des 3 conditions de validité d’une ANOVA. L’objectif est de s’assurer que les résidus suivent une loi normale afin de ne pas affirmer qu’il existe une différence de moyenne entre les groupes qui serait causée par le hasard.

On utilise le test de Shapiro-Wilk pour tester la normalité des résidus où :

* H0 : Les résidus suivent une loi normale si p-value > 5%
* H1 : Les résidus ne suivent pas une loi normale si p-value < 5%

In [ ]:
from scipy.stats import shapiro
model = ols('Calories ~ Type', data=df).fit()
shapiro(model.resid)

* ~H0 : Les résidus suivent une loi normale si p-value > 5%~
* H1 : Les résidus ne suivent pas une loi normale si p-value < 5%

Néanmoins, les conclusions dépendent également tu risques qu'on souhaite. Si on veut  1% de chance de se tromper, alors on ne rejette pas H0 car la p-value > 1%

### Cas de variances égales entre chaque groupe

In [ ]:
A = pd.Series(np.linspace(1,10,9), name='A')
B = pd.Series(np.linspace(31,40,9), name='B')
C = pd.Series(np.linspace(51,60,9), name='C')
Groupe = pd.Series(['A', 'B', 'C']).repeat(9).to_list()

frame = { 'Groupe': Groupe, 'Valeur': pd.concat([A, B, C]) } 
result = pd.DataFrame(frame) 
plt.subplots(figsize=(20,4))
ax = sns.boxplot(x="Valeur", y="Groupe", data=result)

On s'interesse au variance de chaque groupe

In [ ]:
result.groupby("Groupe")['Valeur'].agg('var')

Le test de bartlett permet de tester si les variances sont significativement différentes ou non

In [ ]:
from scipy.stats import bartlett
bartlett(A, B, C)

H0 : Les variances de chaque groupe sont égales si p-value > 5%
<br> ~H1 : Les variances de chaque groupe ne sont pas toutes égales < 5%~
<br> On peut donc faire une ANOVA

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

A travers l'analyse de la variance on cherche à déterminer si : <br>
H0 : Les moyennes de chaque groupe sont égales si p-value > 5%
<br> H1 : Les moyennes de chaque groupe ne sont pas toutes égales < 5%

In [ ]:
model = ols('Valeur ~ Groupe', data=result).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

~H0 : Les moyennes de chaque groupe sont égales si p-value > 5%~
<br> H1 : Les moyennes de chaque groupe ne sont pas toutes égales < 5%

### Cas de variances inégales entre chaque groupe

In [ ]:
A = pd.Series(np.linspace(1,7,9), name='A')
B = pd.Series(np.linspace(31,50,9), name='B')
C = pd.Series(np.linspace(50,100,9), name='C')
Groupe = pd.Series(['A', 'B', 'C']).repeat(9).to_list()

frame = { 'Groupe': Groupe, 'Valeur': pd.concat([A, B, C]) } 
result = pd.DataFrame(frame) 
plt.subplots(figsize=(20,4))
ax = sns.boxplot(x="Valeur", y="Groupe", data=result)

In [ ]:
bartlett(A, B, C)

~H0 : Les variances de chaque groupe sont égales si p-value > 5%~
<br> H1 : Les variances de chaque groupe ne sont pas toutes égales < 5%
<br> Il n'est donc pas conseillé de réaliser une ANOVA car les résultats ne seraient pas fiables.

## Anova à 2 facteurs

Même principe que l'Anova à un facteur sauf qu'on ajoute un autre facteur. L'idée est de tester l'indépendance de ces facteurs sur une variable quantitative continue 

On utilise le dataset ToothGrowth disponible en [cliquant ici](https://github.com/asardell/statistique-python/tree/master/Dataset). On étudie la longueur des odontoblastes (cellules responsables de la croissance dentaire) chez 60 cobayes. Chaque animal a reçu l'une des trois doses de vitamine C (0,5, 1 et 2 mg / jour) par l'une des deux méthodes d'administration, du jus d'orange ou de l'acide ascorbique (une forme de vitamine C et codée VC) :

* len : lLongueur de la dent
* supp : supplément (VC ou OJ).
* dose : dose en milligrammes / jour

In [ ]:
df = pd.read_csv("../Dataset/ToothGrowth.csv")
df.head(3)

#### On étudie la variable supp

In [ ]:
df.supp.unique()

In [ ]:
plt.subplots(figsize=(20,2))
ax = sns.boxplot(x="len", y="supp", data=df)

In [ ]:
bartlett(df.len[df.supp == 'VC'],
        df.len[df.supp == 'OJ'])

H0 : Les variances de chaque groupe sont égales si p-value > 5%
<br> ~H1 : Les variances de chaque groupe ne sont pas toutes égales < 5%~

#### On étudie la variable dose

In [ ]:
df.dose.unique()

In [ ]:
df.dose = df.dose.astype('category')

In [ ]:
plt.subplots(figsize=(20,2))
ax = sns.boxplot(x="len", y="dose", data=df)

In [ ]:
bartlett(df.len[df.dose == "0.5"],
        df.len[df.dose == "1.0"],
        df.len[df.dose == "2.0"])

H0 : Les variances de chaque groupe sont égales si p-value > 5%
<br> ~H1 : Les variances de chaque groupe ne sont pas toutes égales < 5%~

#### On peut faire une ANOVA

H0 : Les moyennes de chaque groupe sont égales si p-value > 5%
<br> H1 : Les moyennes de chaque groupe ne sont pas toutes égales < 5%

In [ ]:
model = ols('len ~ supp + dose', data=df).fit()

In [ ]:
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

📢 Le principe de l'Anova à plusieurs facteurs c'est justement de pouvoir observer les intéractions entre les variables

In [ ]:
model = ols('len ~ supp + dose + supp:dose', data=df).fit()

anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

📢 On voit donc qu'il existe une intéraction entre les deux variables. Pour mesurer quelles associations sont significativement différentes des autres, on peut utilise un test de Tukey qui consiste à faire des tests de comparaison de moyenne sur deux échantillon avec toutes les combinaisons d'association

Pour cela, on crée une colonne avec les combinaisons des deux facteurs.

In [ ]:
df['combinaison'] = df['supp'] + '-' + df['dose'].astype('str')
df.head(3)

In [ ]:
import statsmodels.stats.multicomp as multi 
Results = multi.MultiComparison(df['len'], df['combinaison'])
Results = Results.tukeyhsd()
print(Results)